In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
from datetime import datetime
import subprocess
from dateutil.relativedelta import relativedelta
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import tkinter as tk
import calendar
import time
import locale
import pandas as pd
import shutil
import os

locale.setlocale(locale.LC_TIME, 'pt_BR.UTF-8')

def iniciar_navegador():
    """
    Inicializa e conecta o Selenium a uma instância do navegador Chrome já aberta em modo de depuração remota.

    Returns:
        tuple: (webdriver.Chrome, subprocess.Popen) - O WebDriver e o processo do Chrome.
    """
    try:
        chrome_path = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
        
        # Verificar se o Chrome está instalado
        if not os.path.exists(chrome_path):
            raise FileNotFoundError(f"Chrome não encontrado em: {chrome_path}")
        
        # Iniciar o Chrome em modo debug
        chrome_proc = subprocess.Popen([
            chrome_path,
            "--remote-debugging-port=9222",
            r'--user-data-dir=C:\temp\chromeprofile'
        ])
        
        # Aguardar o Chrome inicializar
        time.sleep(5)
        
        # Configurar opções do Chrome
        options = Options()
        options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        
        # Tentar diferentes abordagens para o ChromeDriver
        try:
            # Primeiro: tentar com ChromeDriverManager
            service = Service(ChromeDriverManager().install())
            navegador = webdriver.Chrome(service=service, options=options)
        except Exception as e:
            print(f"Erro com ChromeDriverManager: {e}")
            # Segundo: tentar sem especificar service (usa PATH)
            try:
                navegador = webdriver.Chrome(options=options)
            except Exception as e2:
                print(f"Erro sem service: {e2}")
                # Terceiro: reinstalar ChromeDriver
                print("Tentando reinstalar ChromeDriver...")
                # Limpar cache do ChromeDriverManager
                import shutil
                cache_dir = os.path.expanduser("~/.wdm")
                if os.path.exists(cache_dir):
                    shutil.rmtree(cache_dir)
                
                service = Service(ChromeDriverManager().install())
                navegador = webdriver.Chrome(service=service, options=options)
        
        navegador.maximize_window()
        navegador.get("https://fusion.fiemg.com.br/fusion/portal")
        return navegador, chrome_proc
        
    except Exception as e:
        print(f"Erro ao inicializar navegador: {e}")
        # Fechar processo do Chrome se foi criado
        try:
            chrome_proc.terminate()
        except:
            pass
        raise

def clicar_elemento(nav, elemento, tipo,automacao_fusion_instance):
    """
    Localiza e clica em um elemento na página web utilizando JavaScript.

    Args:
        nav (WebDriver): O navegador (WebDriver) usado para interagir com a página.
        elemento (str): O seletor do elemento a ser localizado na página.
        tipo (By): O tipo de seletor (e.g., By.ID, By.CLASS_NAME, etc.) usado para localizar o elemento.

    Functionality:
        - Tenta localizar o elemento especificado utilizando `WebDriverWait`, aguardando até 30 segundos para ele aparecer.
        - Clica no elemento utilizando um comando JavaScript para garantir a execução do clique.
        - Se o elemento não for encontrado após o tempo de espera, exibe um alerta através de uma janela Tkinter, informando o usuário para interagir manualmente.

    Returns:
        None: A função tenta localizar e clicar no elemento, e em caso de falha, exibe uma mensagem de alerta ao usuário.

    Raises:
        Exibe um alerta ao usuário se o elemento não for encontrado dentro do tempo de espera.
    """
    while True:
        try:
            obj = WebDriverWait(nav, 10).until(EC.presence_of_element_located((tipo, elemento)))  # Aguarda 10 segundos até o elemento carregar
            nav.execute_script("arguments[0].click();", obj)  # Clica no objeto utilizando JavaScript
            break  # Sai do loop se o comando for bem-sucedido
        except Exception:
            if not automacao_fusion_instance.handle_custom_messagebox_response():
                break

def acessar_iframe(nav, tempo_espera,automacao_fusion_instance, timeout=10):
    """
    Retorna ao conteúdo principal da página (fora de qualquer iframe) e acessa novamente um iframe.

    Args:
        nav (WebDriver): O navegador (WebDriver) usado para interagir com a página.
        tempo_espera (int, float): Tempo, em segundos, para aguardar antes de retornar ao conteúdo padrão.
        timeout (int, optional): Tempo máximo, em segundos, para aguardar o iframe aparecer. O padrão é 10 segundos.

    Functionality:
        - Aguarda o tempo especificado (tempo_espera) antes de trocar o contexto para o conteúdo padrão da página.
        - Muda o contexto do WebDriver para o conteúdo principal (fora de qualquer iframe).
        - Espera até o `timeout` para que o iframe esteja presente no DOM.
        - Muda o contexto do WebDriver para o iframe localizado.

    Returns:
        None: A função realiza a troca de contexto para o iframe, sem retornar um valor.
    """
    while True:
        try:
            time.sleep(tempo_espera)  # Espera antes de mudar para o conteúdo padrão
            nav.switch_to.default_content()  # Volta ao conteúdo principal da página
            iframe = WebDriverWait(nav, timeout).until(EC.presence_of_element_located((By.TAG_NAME, 'iframe')))  # Espera até que o iframe esteja presente
            nav.switch_to.frame(iframe)  # Troca para o iframe
            break  # Sai do loop se o comando for bem-sucedido
        except Exception:
            if not automacao_fusion_instance.handle_custom_messagebox_response():
                break

def clicar_entidade_por_nome(nav, nome, automacao_fusion_instance=None, timeout=15):
    """
    Clica no <span> que contém o texto (pode ter HTML interno devido a ng-bind-html).
    Usa contains(normalize-space(.), nome) para casar mesmo com marcação interna.
    Retorna True se clicou, False caso contrário.
    """
    xpath = f"//span[contains(@class,'ng-binding') and contains(normalize-space(.), \"{nome}\")]"
    while True:
        try:
            el = WebDriverWait(nav, timeout).until(EC.presence_of_element_located((By.XPATH, xpath)))
            nav.execute_script("arguments[0].scrollIntoView({block: 'center'}); arguments[0].click();", el)
            return True
        except Exception:
            if automacao_fusion_instance is not None:
                if not automacao_fusion_instance.handle_custom_messagebox_response():
                    return False
            else:
                resp = input(f"Elemento com texto '{nome}' não encontrado/clicável. Clique manualmente e pressione Enter para tentar novamente (ou 's' para sair): ")
                if resp.strip().lower() == 's':
                    return False
                # tenta novamente

def enviarkey_elemento(nav, elemento, tipo, texto, automacao_fusion_instance):
    """
    Localiza um elemento na página web, limpa o campo e envia um texto para ele.
    """
    while True:
        try:
            # aguarda até o elemento estar clicável e obtém a referência
            elem = WebDriverWait(nav, 10).until(EC.element_to_be_clickable((tipo, elemento)))
            # tenta limpar via Selenium; se falhar, limpa via JS
            try:
                elem.clear()
            except Exception:
                nav.execute_script("arguments[0].value = '';", elem)
                nav.execute_script("arguments[0].dispatchEvent(new Event('input', { bubbles: true }));", elem)
                nav.execute_script("arguments[0].dispatchEvent(new Event('change', { bubbles: true }));", elem)
            # envia o texto
            elem.send_keys(texto)
            # dispara evento change/input para frameworks reativos
            try:
                nav.execute_script("arguments[0].dispatchEvent(new Event('input', { bubbles: true })); arguments[0].dispatchEvent(new Event('change', { bubbles: true }));", elem)
            except Exception:
                pass
            break  # sucesso
        except Exception:
            if not automacao_fusion_instance.handle_custom_messagebox_response():
                break

def esperar_e_pegar_texto(nav, elemento_id, timeout=10, automacao_fusion_instance=None):
    """
    Espera até que um elemento com o ID especificado apareça e retorna seu texto visível.
    Para inputs hidden que têm o texto visível como nó de texto do pai, retorna o texto do pai
    (removendo valores de inputs filhos). Retorna None se o usuário cancelar via automacao_fusion_instance.
    """
    while True:
        try:
            el = WebDriverWait(nav, timeout).until(EC.presence_of_element_located((By.ID, elemento_id)))
            if el is None:
                return " "
            # garantir referência atual e tratar StaleElementReference
            try:
                el = nav.find_element(By.ID, elemento_id)
            except StaleElementReferenceException:
                time.sleep(0.05)
                el = nav.find_element(By.ID, elemento_id)

            # tentar obter o texto visível que pode estar no pai (ex.: <input hidden> + texto no parent)
            try:
                text = nav.execute_script("""
                    var el = arguments[0];
                    if (!el) return '';
                    var p = el.parentNode;
                    if (p) {
                        var txt = p.textContent || '';
                        // remover valores de inputs/textarea/select dentro do pai para não retornar ids/values
                        var controls = p.querySelectorAll('input, textarea, select');
                        controls.forEach(function(c){ if(c.value) txt = txt.replace(c.value, ''); });
                        return txt.trim();
                    }
                    return (el.textContent || el.value || '').trim();
                """, el)
                if text:
                    return text
            except Exception:
                pass

            # fallback: priorizar atributo 'value' apenas se for realmente informativo
            try:
                val = el.get_attribute("value")
                if val is not None and str(val).strip() != "":
                    # se value for só um id e não queremos, não retornar aqui — mas manter como fallback
                    return str(val).strip()
            except Exception:
                pass

            # último recurso: texto do próprio elemento
            try:
                return el.text.strip()
            except Exception:
                return " "
        except Exception as e:
            # se timeout -> retornar espaço em branco conforme pedido
            if isinstance(e, TimeoutException):
                return " "
            if automacao_fusion_instance is not None:
                if not automacao_fusion_instance.handle_custom_messagebox_response():
                    return None
                # se True, repete o loop
            else:
                # comportamento anterior: raise, mas para evitar travar em busca simples,
                # retornar espaço em branco quando não houver handler (conforme requisito de timeout)
                return " "
            
def pegar_texto_com_quebras(nav, elemento_id, timeout=1, automacao_fusion_instance=None):

    """
    Retorna o texto do elemento preservando quebras de linha (<br>) como '\n'
    e decodificando entidades HTML (ex.: &amp; -> &).
    """
    while True:
        try:
            el = WebDriverWait(nav, timeout).until(EC.presence_of_element_located((By.ID, elemento_id)))
            if el is None:
                return " "
            # garantir referência atual e tratar StaleElementReference
            try:
                el = nav.find_element(By.ID, elemento_id)
            except StaleElementReferenceException:
                time.sleep(0.05)
                el = nav.find_element(By.ID, elemento_id)

            # tentar obter o texto visível que pode estar no pai (ex.: <input hidden> + texto no parent)
            try:
                text = nav.execute_script("""
                    var el = arguments[0];
                    if (!el) return '';
                    var p = el.parentNode;
                    if (p) {
                        var txt = p.textContent || '';
                        // remover valores de inputs/textarea/select dentro do pai para não retornar ids/values
                        var controls = p.querySelectorAll('input, textarea, select');
                        controls.forEach(function(c){ if(c.value) txt = txt.replace(c.value, ''); });
                        return txt.trim();
                    }
                    return (el.textContent || el.value || '').trim();
                """, el)
                if text:
                    return text
            except Exception:
                pass

            # fallback: priorizar atributo 'value' apenas se for realmente informativo
            try:
                val = el.get_attribute("value")
                if val is not None and str(val).strip() != "":
                    return str(val).strip()
            except Exception:
                pass

            # último recurso: texto do próprio elemento
            try:
                return el.text.strip()
            except Exception:
                return " "
        except Exception as e:
            # se timeout -> retornar espaço em branco conforme pedido
            if isinstance(e, TimeoutException):
                return " "
            if automacao_fusion_instance is not None:
                if not automacao_fusion_instance.handle_custom_messagebox_response():
                    return None
                # se True, repete o loop
            else:
                # comportamento anterior: raise, mas para evitar travar em busca simples,
                # retornar espaço em branco quando não houver handler (conforme requisito de timeout)
                return " "
            
def extrair_linhas_tabela(nav, tabela_id, timeout=10, automacao_fusion_instance=None):
    """
    Extrai as linhas de uma tabela (identificada pelo id) e retorna lista de strings,
    cada string com os valores das células separados por ';'.
    Ignora a 4ª coluna (index 3) conforme solicitado.
    Em caso de TimeoutException retorna [" "].
    """
    from selenium.common.exceptions import TimeoutException, StaleElementReferenceException

    while True:
        try:
            tabela = WebDriverWait(nav, timeout).until(EC.presence_of_element_located((By.ID, tabela_id)))
            if tabela is None:
                return [" "]

            js = """
                var table = arguments[0];
                var rows = [];
                if (!table) return rows;
                var trs = table.querySelectorAll('tr');
                trs.forEach(function(tr){
                    var style = window.getComputedStyle(tr);
                    if (style && (style.display === 'none' || style.visibility === 'hidden')) return;
                    var cells = tr.querySelectorAll('td, th');
                    if (!cells.length) return;
                    var parts = [];
                    for (var i = 0; i < cells.length; i++) {
                        // ignorar a 4ª coluna (index 3)
                        if (i === 3) continue;
                        var c = cells[i];
                        // prioridade: se houver span com title, usar title (pega texto completo do tooltip)
                        var spanWithTitle = c.querySelector('span[title]');
                        if (spanWithTitle && spanWithTitle.getAttribute('title')) {
                            parts.push(spanWithTitle.getAttribute('title').trim().replace(/\\s+/g, ' '));
                            continue;
                        }
                        var html = c.innerHTML || '';
                        // preservar <br> como quebra de linha
                        html = html.replace(/<br\\s*\\/?>/gi, '\\n');
                        var tmp = document.createElement('div');
                        tmp.innerHTML = html;
                        // remover valores de inputs/textarea/select dentro da célula
                        var controls = tmp.querySelectorAll('input, textarea, select');
                        controls.forEach(function(ctrl){
                            if (ctrl.value) {
                                tmp.innerHTML = tmp.innerHTML.split(ctrl.value).join('');
                            }
                        });
                        var text = (tmp.textContent || tmp.innerText || '').replace(/\\s+/g, ' ').trim();
                        parts.push(text);
                    }
                    // se todas as colunas foram ignoradas (ex.: só 4 cols e 4ª era a única válida), ainda retorna string vazia
                    rows.push(parts.join(';'));
                });
                return rows;
            """
            linhas = nav.execute_script(js, tabela)
            if not linhas:
                return [" "]
            return linhas

        except Exception as e:
            if isinstance(e, TimeoutException):
                return [" "]
            if automacao_fusion_instance is not None:
                if not automacao_fusion_instance.handle_custom_messagebox_response():
                    return None
            else:
                return [" "]


In [2]:
navegador, chrome_proc = iniciar_navegador()

In [ ]:
def fazer_requisicao_wfprocess(navegador, setor, data_busca, offset=0, range_size=100):
    """
    Faz requisição direta ao endpoint WFProcess usando sessão do navegador.
    
    Args:
        navegador: Instância do Selenium WebDriver (navegador logado)
        setor: Nome do setor ("Compras", "Financeiro", "Patrimônio" ou "Regularidade")
        data_busca: Data no formato DD/MM/YYYY
        offset: Posição inicial (paginação)
        range_size: Quantidade de itens por requisição
    
    Returns:
        dict: Dicionário {code: id} ou None em caso de erro
    """
    import requests
    from datetime import datetime
    
    # ✅ CONFIGURAÇÕES POR SETOR
    config_setores = {
        "Compras": {
            "neoId": 361179933,
            "filter_neoId": 366063102,
            "filter_ids": [366063103, 366063104, 366063105],
            "loperand2": 80823850
        },
        "Financeiro": {
            "neoId": 361180049,
            "filter_neoId": 366063106,
            "filter_ids": [366063107, 366063108, 366063109],
            "loperand2": 145984
        },
        "Patrimônio": {
            "neoId": 361180097,
            "filter_neoId": 366063110,
            "filter_ids": [366063111, 366063112, 366063113],
            "loperand2": 44516
        },
        "Regularidade": {
            "neoId": 361180230,
            "filter_neoId": 366063134,
            "filter_ids": [366063135, 366063136, 366063137],
            "loperand2": 40886
        }
    }
    
    # Validar setor
    if setor not in config_setores:
        print(f"❌ Setor '{setor}' inválido. Use: Compras, Financeiro, Patrimônio ou Regularidade")
        return None
    
    config = config_setores[setor]
    
    # ✅ CAPTURAR COOKIES E HEADERS DO NAVEGADOR
    selenium_cookies = navegador.get_cookies()
    
    session = requests.Session()
    for cookie in selenium_cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    
    headers = {
        'User-Agent': navegador.execute_script("return navigator.userAgent;"),
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'pt-BR,pt;q=0.9',
        'Content-Type': 'application/json',
        'Referer': 'https://fusion.fiemg.com.br/fusion/portal',
        'X-Requested-With': 'XMLHttpRequest'
    }
    
    # ✅ CONVERTER DATA DD/MM/YYYY -> ISO 8601
    data_obj = datetime.strptime(data_busca, "%d/%m/%Y")
    data_iso = data_obj.strftime("%Y-%m-%dT03:00:00.000Z")
    
    # ✅ URL E PAYLOAD DINÂMICO
    url = "https://fusion.fiemg.com.br/fusion/services/process/advancedSearch/WFProcess"
    
    payload = {
        "offset": offset,
        "range": range_size,
        "entityFilter": {
            "neoId": config["neoId"],
            "entityName": "WFProcess",
            "name": f"Núcleo - {setor}",
            "description": None,
            "userDefault": False,
            "filter": {
                "neoType": "Fpersist.QLGroupFilter",
                "neoId": config["filter_neoId"],
                "@id": config["filter_neoId"],
                "operator": "and",
                "filterList": [
                    {
                        "neoType": "Fpersist.QLEqualsFilter",
                        "neoId": config["filter_ids"][0],
                        "@id": config["filter_ids"][0],
                        "operand1": "model.versionControl.proxy",
                        "operand2Type": "NeoObject",
                        "operator": "=",
                        "loperand2": 315083955
                    },
                    {
                        "neoType": "Fpersist.QLOpFilter",
                        "neoId": config["filter_ids"][1],
                        "operand1": "startDate",
                        "operator": ">=",
                        "operand2Type": "Date",
                        "doperand2": data_iso
                    },
                    {
                        "neoType": "Fpersist.QLEqualsFilter",
                        "neoId": config["filter_ids"][2],
                        "@id": config["filter_ids"][2],
                        "operand1": "taskSet.taskList.user",
                        "operand2Type": "NeoObject",
                        "operator": "=",
                        "loperand2": config["loperand2"]
                    }
                ]
            },
            "@type": "EntityFilter"
        },
        "entityTypeName": "WFProcess",
        "groupBy": "STARTDATE",
        "order": "desc",
        "formFieldSelected": [],
        "processSelected": 315083955,
        "haveFormFieldSelected": False,
        "haveActivityModelSelected": False,
        "haveManagerSelected": False,
        "haveParticipantsSelected": False,
        "showSubProcess": False,
        "userCode": "trduarte",
        "ignoreFiltersEmpty": False,
        "managedUsers": [205270532],
        "allUsersSelected": False
    }
    
    # ✅ FAZER REQUISIÇÃO
    try:
        print(f"📤 [{setor}] Requisição: offset={offset}, range={range_size}")
        
        response = session.post(url, json=payload, headers=headers, timeout=60)
        
        print(f"📥 Status: {response.status_code}")
        
        if response.status_code == 200:
            data = response.json()
            
            # ✅ CRIAR DICIONÁRIO {CODE: ID}
            dict_processos = {}
            
            for data_grupo, processos_grupo in data.items():
                for process_id_str, process_data in processos_grupo.items():
                    code = process_data.get('code')
                    process_id = process_data.get('id')
                    
                    if code and process_id:
                        dict_processos[code] = process_id
            
            print(f"✅ Obtidos {len(dict_processos)} processos")
            return dict_processos
        else:
            print(f"❌ Erro HTTP {response.status_code}")
            print(f"Resposta: {response.text[:300]}")
            return None
            
    except Exception as e:
        print(f"❌ Erro na requisição: {e}")
        return None

In [15]:
# Testar requisição
dict_processos = fazer_requisicao_wfprocess(
    navegador=navegador,
    setor="Compras",
    data_busca="01/11/2025",
    offset=0,
    range_size=100
)

if dict_processos:
    print(f"\n📊 Total de processos: {len(dict_processos)}")
    
    # Mostrar primeiros 5
    for code, process_id in list(dict_processos.items()):
        print(f"  - Code: {code} → ID: {process_id}")

📤 Requisição: offset=0, range=100
📥 Status: 200
✅ Obtidos 100 processos

📊 Total de processos: 100
  - Code: 004973 → ID: 366003102
  - Code: 004971 → ID: 365958082
  - Code: 004966 → ID: 365904280
  - Code: 004961 → ID: 365889130
  - Code: 004960 → ID: 365887863
  - Code: 004952 → ID: 365837438
  - Code: 004942 → ID: 365780549
  - Code: 004941 → ID: 365774971
  - Code: 004940 → ID: 365768624
  - Code: 004937 → ID: 365749057
  - Code: 004936 → ID: 365742901
  - Code: 004934 → ID: 365740044
  - Code: 004931 → ID: 365661650
  - Code: 004930 → ID: 365658551
  - Code: 004928 → ID: 365653506
  - Code: 004927 → ID: 365640694
  - Code: 004926 → ID: 365637567
  - Code: 004925 → ID: 365633867
  - Code: 004923 → ID: 365630137
  - Code: 004922 → ID: 365616683
  - Code: 004920 → ID: 365609332
  - Code: 004919 → ID: 365605136
  - Code: 004917 → ID: 365597064
  - Code: 004916 → ID: 365595462
  - Code: 004913 → ID: 365571998
  - Code: 004911 → ID: 365519676
  - Code: 004909 → ID: 365515345
  - Code: 

In [3]:
input("Realize o login no portal e pressione Enter para continuar...")
navegador.switch_to.default_content()  # Volta ao conteúdo principal da página
clicar_elemento(navegador, '//*[@id="menu_item_small_30"]/div/div', By.XPATH, automacao_fusion_instance=None)
clicar_elemento(navegador, '#menu_item_expanded_301 > div > div.menu-inner-cell.menu-inner-cell-middle > p', By.CSS_SELECTOR, automacao_fusion_instance=None)
acessar_iframe(navegador, 2, automacao_fusion_instance=None, timeout=15)
clicar_entidade_por_nome(navegador, "Núcleo - Compras", automacao_fusion_instance=None)
enviarkey_elemento(navegador, "input.form-control.ng-valid-date", By.CSS_SELECTOR, "01/10/2025", automacao_fusion_instance=None)
time.sleep(2)  # Pequena pausa para garantir que o campo foi preenchido
clicar_elemento(navegador, 'advancedSearchBtn', By.ID, automacao_fusion_instance=None)


In [3]:
import requests
import json

def capturar_cookies_e_headers(navegador):
    """
    Captura cookies e headers do navegador logado para usar em requests.
    """
    # Pegar todos os cookies da sessão atual
    selenium_cookies = navegador.get_cookies()
    
    # Converter para formato do requests
    session = requests.Session()
    for cookie in selenium_cookies:
        session.cookies.set(cookie['name'], cookie['value'])
    
    # Headers comuns para o Fusion
    headers = {
        'User-Agent': navegador.execute_script("return navigator.userAgent;"),
        'Accept': 'application/json, text/plain, */*',
        'Accept-Language': 'pt-BR,pt;q=0.9',
        'Referer': 'https://fusion.fiemg.com.br/fusion/portal',
        'X-Requested-With': 'XMLHttpRequest'
    }
    
    return session, headers

# Usar após o login:
session, headers = capturar_cookies_e_headers(navegador)

def fazer_requisicao_fusion(session, headers, form_id):
    """
    Faz requisição AJAX para o Fusion e retorna o response.
    
    Args:
        session: requests.Session com cookies
        headers: dict com headers
        form_id: ID do formulário (ex: 363321568)
    
    Returns:
        dict com response_data e status_code
    """
    url = f"https://fusion.fiemg.com.br/fusion/portal/ajaxRender/Form"
    
    params = {
        'content': 'true',
        'id': form_id,
        'showContainer': 'false',
        'disableFooter': 'true',
        'full': 'true',
        'edit': 'false',
        'type': 'DSSNAFPDadosDaSolicitacao'
    }
    
    try:
        response = session.get(url, params=params, headers=headers, timeout=30)
        
        print(f"Status Code: {response.status_code}")
        print(f"Content-Type: {response.headers.get('Content-Type')}")
        
        # Tentar parsear como JSON
        try:
            data = response.json()
            return {
                'success': True,
                'status_code': response.status_code,
                'data': data,
                'raw_text': response.text[:500]  # primeiros 500 chars
            }
        except:
            # Se não for JSON, retornar HTML/texto
            return {
                'success': True,
                'status_code': response.status_code,
                'data': None,
                'raw_text': response.text[:500],
                'full_html': response.text
            }
    
    except Exception as e:
        return {
            'success': False,
            'error': str(e)
        }

def salvar_resposta_em_txt(resultado, caminho_arquivo='resposta_fusion.txt'):
    """
    Salva o conteúdo completo da resposta em um arquivo TXT para análise.
    
    Args:
        resultado: dict retornado pela função fazer_requisicao_fusion
        caminho_arquivo: caminho do arquivo TXT a ser criado
    """
    with open(caminho_arquivo, 'w', encoding='utf-8') as f:
        f.write("=" * 80 + "\n")
        f.write("ANÁLISE COMPLETA DA RESPOSTA FUSION\n")
        f.write("=" * 80 + "\n\n")
        
        # Informações básicas
        f.write(f"Status Code: {resultado.get('status_code')}\n")
        f.write(f"Success: {resultado.get('success')}\n")
        f.write(f"Timestamp: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        # Se for JSON
        if resultado.get('data'):
            f.write("-" * 80 + "\n")
            f.write("TIPO: JSON\n")
            f.write("-" * 80 + "\n\n")
            f.write(json.dumps(resultado['data'], indent=2, ensure_ascii=False))
        
        # Se for HTML/Texto
        elif resultado.get('full_html'):
            f.write("-" * 80 + "\n")
            f.write("TIPO: HTML/TEXTO COMPLETO\n")
            f.write("-" * 80 + "\n\n")
            f.write(resultado['full_html'])
        
        # Se houver erro
        elif resultado.get('error'):
            f.write("-" * 80 + "\n")
            f.write("ERRO\n")
            f.write("-" * 80 + "\n\n")
            f.write(str(resultado['error']))
        
        f.write("\n\n" + "=" * 80 + "\n")
        f.write("FIM DA ANÁLISE\n")
        f.write("=" * 80 + "\n")
    
    print(f"✅ Resposta salva em: {caminho_arquivo}")
    print(f"📄 Tamanho do arquivo: {os.path.getsize(caminho_arquivo):,} bytes")

# Usar após fazer a requisição:
form_id = "363272199"
resultado = fazer_requisicao_fusion(session, headers, form_id)

# Salvar em TXT
salvar_resposta_em_txt(resultado, f'resposta_fusion_{form_id}.txt')

# Visualizar primeiros 1000 caracteres no console
print("\n📋 PREVIEW (primeiros 1000 chars):")
print(resultado.get('raw_text', resultado.get('full_html', ''))[:1000])

Status Code: 200
Content-Type: text/html;charset=UTF-8
✅ Resposta salva em: resposta_fusion_363272199.txt
📄 Tamanho do arquivo: 462,861 bytes

📋 PREVIEW (primeiros 1000 chars):
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<div id="loading_315083471" class="loadingbox">
            <div
                style="z-index: 999; margin: auto;position: absolute;top: 0;left: 0;bottom: 12px;right: 0;"
                class="preloaderCircular">
                <div class="spinner">
                    <div class="left"></div>
                    <div class="right"></div>
                </div>
            </div>
        </


In [4]:
def obter_entity_id_do_processo(session, headers, process_id):
    """
    Faz requisição para o endpoint BPM e extrai o entityId.
    
    Args:
        session: requests.Session com cookies
        headers: dict com headers
        process_id: ID do processo (ex: '363272199')
    
    Returns:
        str: entityId ou None se não encontrado
    """
    url = f"https://fusion.fiemg.com.br/fusion/rest/bpm/entity/process/wfProcessVo/{process_id}"
    
    try:
        response = session.get(url, headers=headers, timeout=30)
        
        print(f"🔍 Status Code: {response.status_code}")
        
        if response.status_code == 200:
            try:
                data = response.json()
                entity_id = data.get('entityId')
                
                if entity_id:
                    print(f"✅ EntityId encontrado: {entity_id}")
                    return str(entity_id)
                else:
                    print(f"⚠️ EntityId não encontrado na resposta")
                    print(f"📄 Resposta: {data}")
                    return None
                    
            except Exception as e:
                print(f"❌ Erro ao parsear JSON: {e}")
                print(f"📄 Resposta raw: {response.text[:500]}")
                return None
        else:
            print(f"❌ Erro HTTP {response.status_code}: {response.text[:200]}")
            return None
            
    except Exception as e:
        print(f"❌ Erro na requisição: {e}")
        return None

# EXEMPLO DE USO:
process_id = "363500603"  # ID do processo que você tem
entity_id = obter_entity_id_do_processo(session, headers, process_id)

if entity_id:
    print(f"🎯 EntityId obtido: {entity_id}")
    # Agora você pode usar o entity_id para outras requisições
else:
    print("❌ Não foi possível obter o EntityId")

🔍 Status Code: 200
✅ EntityId encontrado: 363500604
🎯 EntityId obtido: 363500604


In [4]:
# Definir o número fixo de itens desejado
numero_fixo = 66  # ALTERE ESTE NÚMERO CONFORME NECESSÁRIO

timeout = 60
start = time.time()

ul = WebDriverWait(navegador, 10).until(
    EC.presence_of_element_located((By.XPATH, "//ul[contains(@class,'groupeditemlist') and contains(@class,'pull-left')]"))
)
li_name = "noHoverSpan"

# Dicionário para armazenar {codigo_processo: id_numerico}
dict_processos = {}

while True:
    items = ul.find_elements(By.CLASS_NAME, li_name)
    cur_count = len(items)
    
    # Verifica se já chegou no número desejado
    if cur_count >= numero_fixo:
        print(f"✅ Número desejado atingido: {cur_count} itens")
        break

    # rolar para o último item visível (ou para o container caso não exista item)
    if items:
        navegador.execute_script("arguments[0].scrollIntoView(false);", items[-1])
    else:
        navegador.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", ul)
    navegador.execute_script("window.scrollBy(0, 200);")
    time.sleep(0.6)

    # Timeout global de segurança
    if time.time() - start > timeout:
        print(f"⚠️ Timeout atingido. Itens carregados: {cur_count}")
        break

total_items = len(ul.find_elements(By.CLASS_NAME, li_name))
print(f"Itens encontrados: {total_items}")

# Extrair IDs e códigos de processo de todos os <li>
li_elements = ul.find_elements(By.XPATH, ".//li[starts-with(@id, 'groupeditem-')]")
for li in li_elements:
    try:
        # Pegar o ID do <li>
        li_id = li.get_attribute("id")
        if not li_id:
            continue
        
        # Remove o prefixo "groupeditem-"
        id_numerico = li_id.replace("groupeditem-", "")
        
        # Buscar o <span> com class "processCode ng-binding" dentro deste <li>
        try:
            process_code_span = li.find_element(By.CSS_SELECTOR, "span.processCode.ng-binding")
            process_code = process_code_span.text.strip()
            
            if process_code:
                # Adicionar ao dicionário: {codigo: id}
                dict_processos[process_code] = int(id_numerico) + 1
        except Exception as e:
            print(f"⚠️  Não foi possível encontrar código do processo para ID {id_numerico}")
            continue
            
    except Exception as e:
        print(f"❌ Erro ao processar item: {str(e)[:50]}")
        continue

print(f"\n✅ Processos extraídos: {len(dict_processos)}")
# Se quiser manter a lista de IDs também (opcional)
lista_ids = list(dict_processos.values())

✅ Número desejado atingido: 67 itens
Itens encontrados: 67

✅ Processos extraídos: 67


In [292]:
lista_historico = []

for i in range(total_items):
    linha_atual = []
    acessar_iframe(navegador, 2, automacao_fusion_instance=None, timeout=15)
    ul = WebDriverWait(navegador, 10).until(
    EC.presence_of_element_located((By.XPATH, "//ul[contains(@class,'groupeditemlist') and contains(@class,'pull-left')]"))
    )
    items = ul.find_elements(By.CLASS_NAME, "noHoverSpan")
    if not items:
        raise Exception("Nenhum item com class 'noHoverSpan' encontrado")
    first = items[i]
    # garantir que está visível e tentar clicar
    navegador.execute_script("arguments[0].scrollIntoView({block:'center'});", first)
    try:
        first.click()
    except Exception:
        navegador.execute_script("arguments[0].click();", first)
    iframe2 = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
    navegador.switch_to.frame(iframe2)
    time.sleep(2)  # Espera para garantir que o conteúdo carregou
    numero_chamado = esperar_e_pegar_texto(navegador, "label_Codigo__", timeout=5, automacao_fusion_instance=None).replace('Código:', '')
    data_inicial = pegar_texto_com_quebras(navegador, "var_DadosDaSolicitacao__Responsavel__data__", timeout=1, automacao_fusion_instance=None).replace('Data.:', '')
    responsavel = esperar_e_pegar_texto(navegador, "var_DadosDaSolicitacao__Responsavel__responsavel__", timeout=1, automacao_fusion_instance=None)
    area_setor = esperar_e_pegar_texto(navegador, "label_DadosDaSolicitacao__Responsavel__areaSetor__", timeout=1, automacao_fusion_instance=None).replace('Área / Setor:', '')
    detalhes_solicitacao = pegar_texto_com_quebras(navegador, "var_DadosDaSolicitacao__DescricaoDaDemanda___view_textarea", timeout=1, automacao_fusion_instance=None).replace('Descrição da Demanda:function textElementChangedvar_DadosDaSolicitacao__DescricaoDaDemanda__(){var targetEl;targetEl = textareavar_DadosDaSolicitacao__DescricaoDaDemanda__;var remaining = 2000 - targetEl.value.length;if(remaining < 0){targetEl.value = targetEl.value.substring(0, 2000);remaining = 2000 - targetEl.value.length;}}', '')
    urgencia_demanda = esperar_e_pegar_texto(navegador, "label_DadosDaSolicitacao__UrgenciaDemanda__", timeout=1, automacao_fusion_instance=None).replace('Urgência da Demanda:', '')
    justificativa_demanda = pegar_texto_com_quebras(navegador, "var_DadosDaSolicitacao__JustificativaDaDefinicaoDeUrgencia___view_textarea", timeout=1, automacao_fusion_instance=None).replace('Justificativa da Definição de Urgência:function textElementChangedvar_DadosDaSolicitacao__JustificativaDaDefinicaoDeUrgencia__(){var targetEl;targetEl = textareavar_DadosDaSolicitacao__JustificativaDaDefinicaoDeUrgencia__;var remaining = 2000 - targetEl.value.length;if(remaining < 0){targetEl.value = targetEl.value.substring(0, 2000);remaining = 2000 - targetEl.value.length;}}', '')
    data_atual_supervisor = pegar_texto_com_quebras(navegador, "var_SupervisorAnalise__DataAtual__", timeout=1, automacao_fusion_instance=None).replace('Data Atual:', '')
    prazo_final = pegar_texto_com_quebras(navegador, "var_SupervisorAnalise__PrazoDeAtendimento__", timeout=1, automacao_fusion_instance=None).replace('Prazo de Atendimento:', '')
    historico_nucleo = extrair_linhas_tabela(navegador, "tblist_NucleoAdministrativoAnalise__Historico__", timeout=1)
    encaminhamento_supervisor = extrair_linhas_tabela(navegador, 'tblist_SupervisorAnalise__HistoricoDeAtendimentoNucleoAdministrativo__', timeout=1)
    if encaminhamento_supervisor == [" "]:
        encaminhamento_supervisor = extrair_linhas_tabela(navegador, 'tblist_SupervisorAnalise__HistoricoDeAtendimento__', timeout=1)
    acao_supervisor = esperar_e_pegar_texto(navegador, "label_SupervisorAnalise__Acao__", timeout=1, automacao_fusion_instance=None).replace('Ação:', '')
    responsavel_nucleo = esperar_e_pegar_texto(navegador, "var_NucleoAdministrativoAnalise__Responsavel__responsavel__", timeout=1, automacao_fusion_instance=None)
    acao_nucleo = esperar_e_pegar_texto(navegador, "label_NucleoAdministrativoAnalise__Acoes__", timeout=1, automacao_fusion_instance=None).replace('Ação:', '')
    lista_historico.append([numero_chamado,data_inicial,responsavel,area_setor,detalhes_solicitacao,urgencia_demanda,justificativa_demanda,data_atual_supervisor,prazo_final,historico_nucleo,encaminhamento_supervisor,acao_supervisor,responsavel_nucleo,acao_nucleo])
    clicar_elemento(navegador, 'task_back_btn', By.CLASS_NAME, automacao_fusion_instance=None)

In [275]:
acessar_iframe(navegador, 2, automacao_fusion_instance=None, timeout=15)
ul = WebDriverWait(navegador, 10).until(
EC.presence_of_element_located((By.XPATH, "//ul[contains(@class,'groupeditemlist') and contains(@class,'pull-left')]"))
)
items = ul.find_elements(By.CLASS_NAME, "noHoverSpan")
if not items:
    raise Exception("Nenhum item com class 'noHoverSpan' encontrado")
first = items[69]
# garantir que está visível e tentar clicar
navegador.execute_script("arguments[0].scrollIntoView({block:'center'});", first)
try:
    first.click()
except Exception:
    navegador.execute_script("arguments[0].click();", first)
iframe2 = WebDriverWait(navegador, 10).until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
navegador.switch_to.frame(iframe2)

In [277]:
encaminhamento_supervisor = esperar_e_pegar_texto(navegador, 'label_SupervisorAnalise__Acao__', timeout=5)
print(encaminhamento_supervisor)
#label_SupervisorAnalise__Acao__

Ação:Refinar (Devolver ao solicitante)


In [3]:
def salvar_lista_historico_xlsx(lista_historico, caminho_arquivo, sheet_name='Planilha1'):
    """
    Salva lista_historico em XLSX com colunas definidas.
    Cada item de lista_historico deve ser uma lista na ordem:
    [numero_chamado,data_inicial,responsavel,area_setor,detalhes_solicitacao,
     urgencia_demanda,justificativa_demanda,data_atual_supervisor,prazo_final,
     historico_nucleo,encaminhamento_supervisor,responsavel_nucleo,acao_nucleo]
    Campos que são listas (ex.: historico_nucleo, encaminhamento_supervisor) serão
    concatenados em uma string separada por " | ".
    """
    import pandas as pd
    from openpyxl import load_workbook
    cols = [
        'numero_chamado','data_inicial','responsavel','uo','detalhes_solicitacao',
        'urgencia_demanda','justificativa_demanda','data_atual_supervisor','prazo_final',
        'encaminhamento_supervisor','acao_supervisor','historico_nucleo','responsavel_nucleo','acao_nucleo', 'sc', 'pedido_protheus'
    ]

    def _cell_to_str(v):
        if v is None:
            return ""
        if isinstance(v, (list, tuple)):
            # juntar cada linha/valor; usar ' | ' para separar registros múltiplos
            return " | ".join(str(x) for x in v)
        return str(v)

    rows = []
    for linha in lista_historico:
        rows.append([_cell_to_str(c) for c in linha])

    df = pd.DataFrame(rows, columns=cols)

    # salvar usando openpyxl engine (mantém novoslines se existirem)
    df.to_excel(caminho_arquivo, index=False, sheet_name=sheet_name, engine='openpyxl')


In [ ]:
from bs4 import BeautifulSoup
import os
import html as html_lib
import uo_dict
import re



def extrair_dados_do_txt(caminho_arquivo):
    """
    Lê um arquivo TXT e extrai os dados usando as mesmas lógicas do Selenium.
    
    Args:
        caminho_arquivo: Caminho para o arquivo TXT
    
    Returns:
        Lista com os dados extraídos na mesma ordem que o Selenium
    """
    try:
        # Ler o arquivo
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            conteudo = f.read()
        
        # Extrair apenas o HTML (após "TIPO: HTML/TEXTO COMPLETO")
        if 'TIPO: HTML/TEXTO COMPLETO' in conteudo:
            html_content = conteudo.split('TIPO: HTML/TEXTO COMPLETO')[1]
            html_content = html_content.split('=' * 80)[0]
        else:
            html_content = conteudo
        
        # Parsear HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # Função auxiliar MELHORADA para buscar por ID ou div pai
        def get_text(element_id, buscar_em_pai=False):
            """
            Busca texto por ID preservando quebras de linha (<br>) como '\n'.
            """
            el = soup.find(id=element_id)
            if not el:
                return " "
            
            # Se precisa buscar no elemento pai (ex: div_Codigo__ -> div.text-wrapper)
            if buscar_em_pai:
                text_wrapper = el.find('div', class_='text-wrapper')
                if text_wrapper:
                    # ✅ CONVERTER <br> EM \n NO HTML INTERNO
                    html_interno = str(text_wrapper)
                    html_interno = html_interno.replace('<br>', '\n').replace('<br/>', '\n').replace('<br />', '\n')
                    # Parsear novamente para pegar o texto
                    temp_soup = BeautifulSoup(html_interno, 'html.parser')
                    texto = temp_soup.get_text()
                    texto = html_lib.unescape(texto)
                    return texto.strip()
                

            if el.name == 'label':
                parent = el.parent
                if parent:
                    # Remover o próprio label e outros labels do parent
                    for label_tag in parent.find_all('label'):
                        label_tag.decompose()
                    # Remover tags indesejadas
                    for tag in parent.find_all(['script', 'style', 'input', 'select', 'textarea', 'img']):
                        tag.decompose()
                    # Converter <br> em \n
                    html_interno = str(parent)
                    html_interno = html_interno.replace('<br>', '\n').replace('<br/>', '\n').replace('<br />', '\n')
                    temp_soup = BeautifulSoup(html_interno, 'html.parser')
                    texto = temp_soup.get_text()
                    texto = html_lib.unescape(texto)
                    return texto.strip()
            
            # Para inputs hidden, buscar texto no parent
            if el.name == 'input' and el.get('type') == 'hidden':
                parent = el.parent
                if parent:
                    html_interno = str(parent)
                    html_interno = html_interno.replace('<br>', '\n').replace('<br/>', '\n').replace('<br />', '\n')
                    temp_soup = BeautifulSoup(html_interno, 'html.parser')
                    parent_text = temp_soup.get_text()
                    input_value = el.get('value', '')
                    if input_value and input_value in parent_text:
                        parent_text = parent_text.replace(input_value, '').strip()
                    parent_text = html_lib.unescape(parent_text)
                    return parent_text.strip()
            
            # Remover scripts, styles, inputs
            for tag in el.find_all(['script', 'style', 'input', 'select', 'textarea']):
                tag.decompose()
            
            # ✅ CONVERTER <br> EM \n NO HTML INTERNO DO ELEMENTO
            html_interno = str(el)
            html_interno = html_interno.replace('<br>', '\n').replace('<br/>', '\n').replace('<br />', '\n')
            
            # Parsear novamente para pegar o texto com as quebras de linha
            temp_soup = BeautifulSoup(html_interno, 'html.parser')
            texto = temp_soup.get_text()
            texto = html_lib.unescape(texto)
            return texto.strip()
        # Função para tabelas (sem mudanças)
        def parse_data_textarea_soup(soup, textarea_id):
            """
            Parseia o conteúdo (XML/CDATA) dentro de <textarea id="data_*"> e retorna
            uma lista com uma string formatada: "Responsavel ; Data ; Mensagem".
            Remove duplicações quando a mensagem aparece duas vezes (texto + tooltip).
            """
            try:
                remover_texto = textarea_id.strip().replace('data_', '')

                ta = soup.find(id=textarea_id)
                if not ta:
                    return [" "]

                xml_content = (ta.string or ta.get_text() or "").strip()
                if not xml_content:
                    return [" "]
                xml_content = html_lib.unescape(xml_content)

                inner = BeautifulSoup(xml_content, "html.parser")

                # RESPONSÁVEL: procurar input hidden com "responsavel" e remover o value do texto do pai
                input_el = inner.find('input', id=lambda x: x and 'responsavel' in x)
                if input_el:
                    parent = input_el.parent or inner
                    val = input_el.get('value', '')
                    parent_text = parent.get_text(separator=' ', strip=True)
                    if val:
                        parent_text = parent_text.replace(val, '')
                    responsavel = parent_text.strip()
                else:
                    td0 = inner.find('td')
                    responsavel = td0.get_text(separator=' ', strip=True) if td0 else ""

                # DATA: procurar span cujo id contenha "data__"
                date_span = inner.find('span', id=lambda x: x and 'data__' in x)
                data = date_span.get_text(strip=True) if date_span else ""

                # MENSAGEM: priorizar span com title (tooltip) ou span id tooltip*
                msg_span = inner.find('span', title=True) or inner.find('span', id=lambda x: x and x.startswith('tooltip'))
                if msg_span:
                    mensagem = msg_span.get('title') or msg_span.get_text(separator=' ', strip=True)
                else:
                    # fallback: buscar texto remanescente excluindo responsavel e data
                    texts = [t.strip() for t in inner.stripped_strings if t.strip()]
                    texts = [t for t in texts if t not in (responsavel, data)]
                    mensagem = texts[0] if texts else ""

                # normalizar entidades e espaços
                responsavel = html_lib.unescape(responsavel).replace('\xa0', ' ').strip().replace(remover_texto, '')
                data = html_lib.unescape(data).replace('\xa0', ' ').strip()
                mensagem = html_lib.unescape(mensagem).replace('\xa0', ' ').strip()

                # remover duplicação exata (ex.: "X...X..." -> "X...")
                def _collapse_double(s):
                    s = s.strip()
                    n = len(s)
                    # procura ponto de divisão onde as duas metades são iguais (caso mais comum)
                    for mid in range(1, n):
                        left = s[:mid].strip()
                        right = s[mid:].strip()
                        if left and left == right:
                            return left
                    return s

                mensagem = _collapse_double(mensagem)

                # garantir que o nome do responsável não contenha a mensagem nem a data
                if mensagem:
                    responsavel = responsavel.replace(mensagem, '').strip()
                if data:
                    responsavel = responsavel.replace(data, '').strip()

                if data:
                    combined = f"{responsavel.split('  ')[0]} ; {data} ; {mensagem}"
                    return [combined]
                return [" "]
            except Exception:
                return [" "]
        
        def get_data_textarea_ids_from_soup(soup, container_div_id=None, allow_global_fallback=True):
            ids = []
            if container_div_id:
                container = soup.find(id=container_div_id)
                if container:
                    ids.extend(
                        ta.get('id')
                        for ta in container.find_all('textarea', id=lambda v: v and v.startswith('data_'))
                    )
            if not ids and allow_global_fallback:
                ids.extend(
                    ta.get('id')
                    for ta in soup.find_all('textarea', id=lambda v: v and v.startswith('data_'))
                )
            return [i for i in ids if i]

        def parse_all_textareas(textarea_ids):
            combined, seen = [], set()
            for ta_id in textarea_ids:
                parsed = parse_data_textarea_soup(soup, ta_id)
                for entry in parsed:
                    entry = entry.strip()
                    if entry and entry not in seen:
                        seen.add(entry)
                        combined.append(entry)
            return combined or [" "]

        # Extrair dados na mesma ordem do Selenium
        numero_chamado = get_text("div_Codigo__", buscar_em_pai=True)  # ✅ BUSCA NO TEXT-WRAPPER
        data_inicial = get_text("var_DadosDaSolicitacao__Responsavel__data__").replace('Data.:', '').replace('Data:', '').strip()
        responsavel = get_text("var_DadosDaSolicitacao__Responsavel__responsavel__")
        detalhes_solicitacao = get_text("var_DadosDaSolicitacao__DescricaoDaDemanda___view_textarea")
        uo = ''
        if detalhes_solicitacao:
            for uo_unidade in uo_dict.uo_dict:
                pattern = rf'(?<!\d){uo_unidade}(?!\d)'
                if re.search(pattern, detalhes_solicitacao):
                    uo = str(uo_unidade)
                    break
        if detalhes_solicitacao == "":
            detalhes_solicitacao = get_text("var_DadosDaSolicitacao__NecessidadeDeCompras__JustificativaDaDefinicaoDeUrgencia___view_textarea", buscar_em_pai=True)
        urgencia_demanda = get_text("label_DadosDaSolicitacao__UrgenciaDemanda__").replace('Urgência da Demanda:', '').strip()
        if urgencia_demanda == "":
            urgencia_demanda = get_text("label_DadosDaSolicitacao__VariacoesDaDemanda__").replace('Urgência da Demanda:', '').strip()
        justificativa_demanda = get_text("var_DadosDaSolicitacao__JustificativaDaDefinicaoDeUrgencia___view_textarea")
        if justificativa_demanda == "":
            justificativa_demanda = get_text("var_DadosDaSolicitacao__NecessidadeDeCompras__JustificativaDaNecessidadeDeCompra___view_textarea", buscar_em_pai=True)
        data_atual_supervisor = get_text("var_SupervisorAnalise__DataAtual__").replace('Data Atual:', '').strip()
        prazo_final = get_text("var_SupervisorAnalise__PrazoDeAtendimento__").replace('Prazo de Atendimento:', '').strip()
        supervisor_ids = get_data_textarea_ids_from_soup(
            soup, 'dlist_SupervisorAnalise__HistoricoDeAtendimentoNucleoAdministrativo__'
        )
        supervisor_ids_alt = get_data_textarea_ids_from_soup(
            soup, 'dlist_SupervisorAnalise__HistoricoDeAtendimento__'
        )
        nucleo_ids = get_data_textarea_ids_from_soup(
            soup, 'dlist_NucleoAdministrativoAnalise__Historico__', allow_global_fallback=False
        )
        historico_nucleo = parse_all_textareas(nucleo_ids)
        encaminhamento_supervisor = parse_all_textareas(supervisor_ids)
        if encaminhamento_supervisor == [" "]:
            encaminhamento_supervisor = parse_all_textareas(supervisor_ids_alt)
        acao_supervisor = get_text("label_SupervisorAnalise__Acao__").replace('Ação:', '').strip()
        responsavel_nucleo = get_text("var_NucleoAdministrativoAnalise__Responsavel__responsavel__")
        acao_nucleo = get_text("label_NucleoAdministrativoAnalise__Acoes__").replace('Ação:', '').strip()
        
        return [
            numero_chamado, data_inicial, responsavel, uo, detalhes_solicitacao,
            urgencia_demanda, justificativa_demanda, data_atual_supervisor, prazo_final,
            encaminhamento_supervisor, acao_supervisor, historico_nucleo, responsavel_nucleo, acao_nucleo, '', ''
        ]
        
    except Exception as e:
        print(f"❌ Erro ao processar {caminho_arquivo}: {str(e)[:100]}")
        return None

# PROCESSAR ARQUIVOS TXT USANDO O DICIONÁRIO
lista_historico = []
arquivos_nao_encontrados = []

print(f"🔍 Processando {len(dict_processos)} arquivos TXT...")
print("-" * 60)

for idx, (processo, id_numerico) in enumerate(dict_processos.items(), 1):
    # Buscar arquivo com ID original
    arquivo = f'resposta_fusion_{id_numerico}.txt'
    
    # Se não encontrar, tentar com ID-1, ID+1, ID+2
    arquivos_possiveis = [
        f'resposta_fusion_{id_numerico}.txt',
        f'resposta_fusion_{id_numerico - 1}.txt',
        f'resposta_fusion_{id_numerico + 1}.txt',
        f'resposta_fusion_{id_numerico + 2}.txt'
    ]
    
    arquivo_encontrado = None
    for arq in arquivos_possiveis:
        if os.path.exists(arq):
            arquivo_encontrado = arq
            break
    
    if not arquivo_encontrado:
        print(f"[{idx}/{len(dict_processos)}] ⚠️  Processo {processo}: Arquivo não encontrado")
        arquivos_nao_encontrados.append(processo)
        continue
    
    dados = extrair_dados_do_txt(arquivo_encontrado)
    
    if dados:
        lista_historico.append(dados)
    else:
        print(f"[{idx}/{len(dict_processos)}] ❌ Processo {processo}: Falha ao processar")

print("\n" + "=" * 60)
print(f"✅ Total processado: {len(lista_historico)}/{len(dict_processos)} processos")

if arquivos_nao_encontrados:
    print(f"\n⚠️  Arquivos não encontrados para {len(arquivos_nao_encontrados)} processos:")
    for proc in arquivos_nao_encontrados:
        print(f"   - {proc}")

# Salvar em Excel
salvar_lista_historico_xlsx(lista_historico, "historico_chamados_completo.xlsx", sheet_name='Histórico Chamados')
print(f"\n💾 Planilha salva: historico_chamados_completo.xlsx")

🔍 Processando 67 arquivos TXT...
------------------------------------------------------------

✅ Total processado: 67/67 processos

💾 Planilha salva: historico_chamados_completo.xlsx


In [ ]:
import random

# realizar request de toda lista com validação por CONTEÚDO
print(f"🔄 Iniciando requisições para {len(dict_processos)} processos...")
print("-" * 60)

requisicoes_extras = 0
erros_totais = 0

for idx, (processo, id_numerico) in enumerate(dict_processos.items(), 1):
    form_id = str(id_numerico)
    
    print(f"[{idx}/{len(dict_processos)}] 📥 Requisitando: {processo} (ID: {form_id})", end='')
    
    # Fazer requisição principal
    resultado = fazer_requisicao_fusion(session, headers, form_id)
    html_completo = resultado.get('full_html', '')
    
    # Validar se contém o código do processo E "Supervisor Responsável"
    tem_processo = processo in html_completo
    tem_supervisor = "Supervisor Responsável" in html_completo
    
    if not (tem_processo and tem_supervisor):
        print(f" ⚠️  Conteúdo incompleto")
        if not tem_processo:
            print(f"       ❌ Código '{processo}' não encontrado")
        if not tem_supervisor:
            print(f"       ❌ 'Supervisor Responsável' não encontrado")
        
        print(f"       🔄 Tentando ID: {int(form_id)-1}")
        
        # Tentativa 1: ID - 1
        form_id_corrigido = str(int(form_id) - 1)
        resultado = fazer_requisicao_fusion(session, headers, form_id_corrigido)
        html_completo = resultado.get('full_html', '')
        
        tem_processo = processo in html_completo
        tem_supervisor = "Supervisor Responsável" in html_completo
        
        if tem_processo and tem_supervisor:
            print(f"       ✅ Conteúdo completo com ID-1")
            salvar_resposta_em_txt(resultado, f'resposta_fusion_{form_id_corrigido}.txt')
            requisicoes_extras += 1
        else:
            # Tentativa 2: ID + 1
            print(f"       ⚠️  Ainda incompleto! Tentando ID: {int(form_id)+1}")
            form_id_corrigido = str(int(form_id) + 1)
            resultado = fazer_requisicao_fusion(session, headers, form_id_corrigido)
            html_completo = resultado.get('full_html', '')
            
            tem_processo = processo in html_completo
            tem_supervisor = "Supervisor Responsável" in html_completo
            
            if tem_processo and tem_supervisor:
                print(f"       ✅ Conteúdo completo com ID+1")
                salvar_resposta_em_txt(resultado, f'resposta_fusion_{form_id_corrigido}.txt')
                requisicoes_extras += 1
            else:
                # Tentativa 3: ID + 2
                print(f"       ⚠️  Ainda incompleto! Tentando ID: {int(form_id)+2}")
                form_id_corrigido = str(int(form_id) + 2)
                resultado = fazer_requisicao_fusion(session, headers, form_id_corrigido)
                html_completo = resultado.get('full_html', '')
                
                tem_processo = processo in html_completo
                tem_supervisor = "Supervisor Responsável" in html_completo
                
                if tem_processo and tem_supervisor:
                    print(f"       ✅ Conteúdo completo com ID+2")
                    salvar_resposta_em_txt(resultado, f'resposta_fusion_{form_id_corrigido}.txt')
                    requisicoes_extras += 1
                else:
                    # TODAS AS TENTATIVAS FALHARAM
                    print(f"       ❌ ERRO: Conteúdo incompleto após 4 tentativas - TXT NÃO SALVO")
                    erros_totais += 1
    else:
        print(f" ✅")
        salvar_resposta_em_txt(resultado, f'resposta_fusion_{form_id}.txt')
    
    time.sleep(random.uniform(0.5, 2.0))

print("\n" + "=" * 60)
print(f"✅ Requisições concluídas!")
print(f"📊 Total de processos: {len(dict_processos)}")
print(f"🔄 Correções bem-sucedidas: {requisicoes_extras}")
print(f"❌ ERROS (sem TXT gerado): {erros_totais}")

if erros_totais > 0:
    print("\n⚠️  ATENÇÃO: Alguns processos NÃO foram salvos!")
    print(f"   Total de arquivos salvos: {len(dict_processos) - erros_totais}")

In [24]:
from bs4 import BeautifulSoup
import os
import html as html_lib
import uo_dict
import re
from concurrent.futures import ThreadPoolExecutor

def extrair_dados_do_txt(caminho_arquivo):
    """
    Lê um arquivo TXT e extrai os dados usando as mesmas lógicas do Selenium.
    
    Args:
        caminho_arquivo: Caminho COMPLETO para o arquivo TXT
    
    Returns:
        Lista com os dados extraídos na mesma ordem que o Selenium
    """
    try:
        # Ler o arquivo
        with open(caminho_arquivo, 'r', encoding='utf-8') as f:
            conteudo = f.read()
        
        # Extrair apenas o HTML (após "TIPO: HTML/TEXTO COMPLETO")
        if 'TIPO: HTML/TEXTO COMPLETO' in conteudo:
            html_content = conteudo.split('TIPO: HTML/TEXTO COMPLETO')[1]
            html_content = html_content.split('=' * 80)[0]
        else:
            html_content = conteudo
        
        # Parsear HTML
        soup = BeautifulSoup(html_content, 'html.parser')
        
        # [TODAS AS FUNÇÕES AUXILIARES GET_TEXT, PARSE_DATA_TEXTAREA_SOUP, ETC. - MANTER IGUAL]
        def get_text(element_id, buscar_em_pai=False):
            el = soup.find(id=element_id)
            if not el:
                return " "
            
            if buscar_em_pai:
                text_wrapper = el.find('div', class_='text-wrapper')
                if text_wrapper:
                    html_interno = str(text_wrapper)
                    html_interno = html_interno.replace('<br>', '\n').replace('<br/>', '\n').replace('<br />', '\n')
                    temp_soup = BeautifulSoup(html_interno, 'html.parser')
                    texto = temp_soup.get_text()
                    texto = html_lib.unescape(texto)
                    return texto.strip()

            if el.name == 'label':
                parent = el.parent
                if parent:
                    for label_tag in parent.find_all('label'):
                        label_tag.decompose()
                    for tag in parent.find_all(['script', 'style', 'input', 'select', 'textarea', 'img']):
                        tag.decompose()
                    html_interno = str(parent)
                    html_interno = html_interno.replace('<br>', '\n').replace('<br/>', '\n').replace('<br />', '\n')
                    temp_soup = BeautifulSoup(html_interno, 'html.parser')
                    texto = temp_soup.get_text()
                    texto = html_lib.unescape(texto)
                    return texto.strip()
            
            if el.name == 'input' and el.get('type') == 'hidden':
                parent = el.parent
                if parent:
                    html_interno = str(parent)
                    html_interno = html_interno.replace('<br>', '\n').replace('<br/>', '\n').replace('<br />', '\n')
                    temp_soup = BeautifulSoup(html_interno, 'html.parser')
                    parent_text = temp_soup.get_text()
                    input_value = el.get('value', '')
                    if input_value and input_value in parent_text:
                        parent_text = parent_text.replace(input_value, '').strip()
                    parent_text = html_lib.unescape(parent_text)
                    return parent_text.strip()
            
            for tag in el.find_all(['script', 'style', 'input', 'select', 'textarea']):
                tag.decompose()
            
            html_interno = str(el)
            html_interno = html_interno.replace('<br>', '\n').replace('<br/>', '\n').replace('<br />', '\n')
            temp_soup = BeautifulSoup(html_interno, 'html.parser')
            texto = temp_soup.get_text()
            texto = html_lib.unescape(texto)
            return texto.strip()

        def parse_data_textarea_soup(soup, textarea_id):
                """
                Parseia o conteúdo de uma textarea específica identificada por seu ID,
                extraindo informações de responsável, data e mensagem.
                Args:
                    soup: Objeto BeautifulSoup do HTML completo
                    textarea_id: ID da textarea a ser parseada
                Returns:
                    Lista com as entradas extraídas no formato "Responsável ; Data ; Mensagem"
                """
                try:
                    ta = soup.find(id=textarea_id)
                    if not ta:
                        return [" "]

                    xml_content = (ta.string or ta.get_text() or "").strip()
                    if not xml_content:
                        return [" "]
                    
                    # ✅ PRIMEIRO UNESCAPE
                    xml_content = html_lib.unescape(xml_content)
                    inner = BeautifulSoup(xml_content, "html.parser")

                    # ✅ BUSCAR E PROCESSAR SEÇÕES CDATA
                    responsavel = ""
                    data = ""
                    mensagem = ""

                    # 1. RESPONSÁVEL: Buscar no primeiro CDATA que contém input hidden
                    cdata_sections = inner.find_all(string=True)
                    for cdata in cdata_sections:
                        cdata_text = str(cdata).strip()
                        
                        # Se contém input hidden com responsavel
                        if 'responsavel' in cdata_text and 'input' in cdata_text:
                            # ✅ SEGUNDO UNESCAPE para decodificar &lt; &gt; &#39; etc.
                            decoded_cdata = html_lib.unescape(cdata_text)
                            temp_soup = BeautifulSoup(decoded_cdata, 'html.parser')
                            
                            # Buscar input hidden
                            input_el = temp_soup.find('input', type='hidden')
                            if input_el:
                                # Pegar texto após o input
                                parent_text = temp_soup.get_text()
                                input_value = input_el.get('value', '')
                                if input_value:
                                    parent_text = parent_text.replace(input_value, '').strip()
                                responsavel = parent_text.strip()
                                break

                    # 2. DATA: Buscar span com id contendo 'data__'
                    for cdata in cdata_sections:
                        cdata_text = str(cdata).strip()
                        
                        if 'data__' in cdata_text and 'span' in cdata_text:
                            # ✅ SEGUNDO UNESCAPE
                            decoded_cdata = html_lib.unescape(cdata_text)
                            temp_soup = BeautifulSoup(decoded_cdata, 'html.parser')
                            
                            # Buscar span de data
                            date_span = temp_soup.find('span', id=lambda x: x and 'data__' in x)
                            if date_span:
                                data = date_span.get_text(strip=True)
                                break

                    # 3. MENSAGEM: Buscar span com title (tooltip)
                    for cdata in cdata_sections:
                        cdata_text = str(cdata).strip()
                        
                        if 'tooltip' in cdata_text and 'title' in cdata_text:
                            # ✅ SEGUNDO UNESCAPE
                            decoded_cdata = html_lib.unescape(cdata_text)
                            temp_soup = BeautifulSoup(decoded_cdata, 'html.parser')
                            
                            # Buscar span com title
                            msg_span = temp_soup.find('span', title=True)
                            if msg_span:
                                mensagem = msg_span.get('title', '')
                                # ✅ TERCEIRO UNESCAPE para a mensagem (pode estar triplo-escapada)
                                mensagem = html_lib.unescape(mensagem)
                                break

                    # 4. BUSCAR NA TAG <overview> como fallback para mensagem
                    if not mensagem:
                        overview_tag = inner.find('overview')
                        if overview_tag:
                            overview_content = overview_tag.get_text(strip=True)
                            if overview_content:
                                # ✅ SEGUNDO UNESCAPE
                                mensagem = html_lib.unescape(overview_content)

                    # Limpeza final
                    responsavel = responsavel.replace(textarea_id.replace('data_', ''), '').strip()
                    data = data.replace('Data.:', '').replace('Data:', '').strip()
                    
                    # Remover duplicações na mensagem
                    def _collapse_double(s):
                        s = s.strip()
                        n = len(s)
                        for mid in range(1, n):
                            left = s[:mid].strip()
                            right = s[mid:].strip()
                            if left and left == right:
                                return left
                        return s

                    mensagem = _collapse_double(mensagem)

                    if data and responsavel:
                        combined = f"{responsavel} ; {data} ; {mensagem}"
                        return [combined]
                    else:
                        return [" "]
                        
                except Exception as e:
                    return [" "]
            
        def get_data_textarea_ids_from_soup(soup, container_div_id=None, allow_global_fallback=True): # Obtém IDs de textareas data_ dentro de um container específico ou globalmente
                ids = [] # Lista para armazenar os IDs encontrados
                if container_div_id: # Se um ID de container for fornecido, buscar dentro dele
                    container = soup.find(id=container_div_id) # Encontrar o container pelo ID
                    if container: # Se o container for encontrado
                        ids.extend( # Extrair IDs de textareas data_ dentro do container
                            ta.get('id') # Obter o ID do textarea
                            for ta in container.find_all('textarea', id=lambda v: v and v.startswith('data_')) # Filtrar textareas com ID começando com 'data_'
                        )
                if not ids and allow_global_fallback: # Se nenhum ID foi encontrado e o fallback global é permitido
                    ids.extend( # Extrair IDs de textareas data_ globalmente
                        ta.get('id') # Obter o ID do textarea
                        for ta in soup.find_all('textarea', id=lambda v: v and v.startswith('data_')) # Filtrar textareas com ID começando com 'data_'
                    ) 
                return [i for i in ids if i] # Retornar a lista de IDs encontrados

        def parse_all_textareas(textarea_ids): # Parseia todas as textareas dadas pelos IDs e combina os resultados, removendo duplicatas
                combined, seen = [], set() # Listas para resultados combinados e conjunto para rastrear vistos
                for ta_id in textarea_ids: # Iterar sobre cada ID de textarea
                    parsed = parse_data_textarea_soup(soup, ta_id) # Parsear o conteúdo da textarea
                    for entry in parsed: # Iterar sobre cada entrada parseada
                        entry = entry.strip() # Remover espaços extras
                        if entry and entry not in seen: # Se a entrada não for vazia e não tiver sido vista antes
                            seen.add(entry) # Marcar como visto
                            combined.append(entry) # Adicionar à lista combinada
                return combined or [" "] # Retornar a lista combinada ou uma lista com espaço se vazia


        # ✅ EXTRAIR SETOR DO NOME DO ARQUIVO
        nome_arquivo = os.path.basename(caminho_arquivo)
        setor = "Desconhecido"  # valor padrão
        
        # Extrair dados
        numero_chamado = get_text("div_Codigo__", buscar_em_pai=True)
        data_inicial = get_text("var_DadosDaSolicitacao__Responsavel__data__").replace('Data.:', '').replace('Data:', '').strip()
        responsavel = get_text("var_DadosDaSolicitacao__Responsavel__responsavel__")
        detalhes_solicitacao = get_text("var_DadosDaSolicitacao__DescricaoDaDemanda___view_textarea")
        
        uo = ''
        if detalhes_solicitacao:
            for uo_unidade in uo_dict.uo_dict:
                pattern = rf'(?<!\d){uo_unidade}(?!\d)'
                if re.search(pattern, detalhes_solicitacao):
                    uo = str(uo_unidade)
                    break
        
        if detalhes_solicitacao == "":
            detalhes_solicitacao = get_text("var_DadosDaSolicitacao__NecessidadeDeCompras__JustificativaDaDefinicaoDeUrgencia___view_textarea", buscar_em_pai=True)
        
        urgencia_demanda = get_text("label_DadosDaSolicitacao__UrgenciaDemanda__").replace('Urgência da Demanda:', '').strip()
        if urgencia_demanda == "":
            urgencia_demanda = get_text("label_DadosDaSolicitacao__VariacoesDaDemanda__").replace('Urgência da Demanda:', '').strip()
        
        justificativa_demanda = get_text("var_DadosDaSolicitacao__JustificativaDaDefinicaoDeUrgencia___view_textarea")
        if justificativa_demanda == "":
            justificativa_demanda = get_text("var_DadosDaSolicitacao__NecessidadeDeCompras__JustificativaDaNecessidadeDeCompra___view_textarea", buscar_em_pai=True)
        
        data_atual_supervisor = get_text("var_SupervisorAnalise__DataAtual__").replace('Data Atual:', '').strip()
        prazo_final = get_text("var_SupervisorAnalise__PrazoDeAtendimento__").replace('Prazo de Atendimento:', '').strip()
        
        supervisor_ids = get_data_textarea_ids_from_soup(soup, 'dlist_SupervisorAnalise__HistoricoDeAtendimentoNucleoAdministrativo__')
        supervisor_ids_alt = get_data_textarea_ids_from_soup(soup, 'dlist_SupervisorAnalise__HistoricoDeAtendimento__')
        nucleo_ids = get_data_textarea_ids_from_soup(soup, 'dlist_NucleoAdministrativoAnalise__Historico__', allow_global_fallback=False)
        
        historico_nucleo = parse_all_textareas(nucleo_ids)
        encaminhamento_supervisor = parse_all_textareas(supervisor_ids)
        if encaminhamento_supervisor == [" "]:
            encaminhamento_supervisor = parse_all_textareas(supervisor_ids_alt)
        
        acao_supervisor = get_text("label_SupervisorAnalise__Acao__").replace('Ação:', '').strip()
        responsavel_nucleo = get_text("var_NucleoAdministrativoAnalise__Responsavel__responsavel__")
        acao_nucleo = get_text("label_NucleoAdministrativoAnalise__Acoes__").replace('Ação:', '').strip()
        nota_atendimento = get_text("var_PesquisaDeSatisfacao__ComoVoceClassificaOAtendimento__")
        justificativa_pesquisa = get_text("var_PesquisaDeSatisfacao__Justificativa___view_textarea")
        
        return [
            numero_chamado, setor, data_inicial, responsavel, uo, detalhes_solicitacao,
            urgencia_demanda, justificativa_demanda, data_atual_supervisor, prazo_final,
            encaminhamento_supervisor, acao_supervisor, historico_nucleo, responsavel_nucleo, acao_nucleo, 
            nota_atendimento, justificativa_pesquisa
        ]
        
    except Exception as e:
        print(f"❌ Erro ao processar {os.path.basename(caminho_arquivo)}: {str(e)[:100]}")
        return None

def processar_pasta_txt(pasta_txt, setor="Desconhecido", max_workers=10):
    """
    Processa TODOS os arquivos TXT de uma pasta em paralelo.
    
    Args:
        pasta_txt: Caminho da pasta contendo os arquivos TXT
        setor: Nome do setor (opcional, usado na planilha final)
        max_workers: Número de threads paralelas
    
    Returns:
        lista_historico: Lista com todos os dados extraídos
    """
    # ✅ LISTAR TODOS OS ARQUIVOS .TXT DA PASTA
    arquivos_txt = [
        os.path.join(pasta_txt, f)
        for f in os.listdir(pasta_txt)
        if f.endswith('.txt') and f.startswith('resposta_fusion_')
    ]
    
    if not arquivos_txt:
        print(f"❌ Nenhum arquivo 'resposta_fusion_*.txt' encontrado em: {pasta_txt}")
        return []
    
    print(f"🔍 Encontrados {len(arquivos_txt)} arquivos TXT")
    print(f"🚀 Processando em paralelo ({max_workers} threads)...")
    print("-" * 60)
    
    # ✅ PROCESSAR EM PARALELO
    lista_historico = []
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        resultados = list(executor.map(extrair_dados_do_txt, arquivos_txt))
    
    # ✅ FILTRAR RESULTADOS VÁLIDOS
    for idx, dados in enumerate(resultados, 1):
        if dados:
            print(f"[{idx}/{len(arquivos_txt)}] ✅ Sucesso: {os.path.basename(arquivos_txt[idx-1])}")
            lista_historico.append(dados)
        else:
            nome_arquivo = os.path.basename(arquivos_txt[idx-1])
            print(f"[{idx}/{len(arquivos_txt)}] ⚠️  Falha: {nome_arquivo}")
    
    print("\n" + "=" * 60)
    print(f"✅ Processamento concluído!")
    print(f"📊 Total processado: {len(lista_historico)}/{len(arquivos_txt)} arquivos")
    
    return lista_historico

# ✅ EXEMPLO 1: Processar pasta e salvar em Excel
pasta_regularidade = r"C:\Users\trduarte\Downloads\automacao_nucleo-main\Financeiro"

lista_historico = processar_pasta_txt(
    pasta_txt=pasta_regularidade,
    setor="Financeiro",
    max_workers=10
)

# Salvar em Excel
if lista_historico:
    salvar_lista_historico_xlsx(
        lista_historico, 
        "historico_financeiro_completo.xlsx", 
        sheet_name='Financeiro'
    )
    print(f"\n💾 Planilha salva: historico_financeiro_completo.xlsx")

🔍 Encontrados 1584 arquivos TXT
🚀 Processando em paralelo (10 threads)...
------------------------------------------------------------
[1/1584] ✅ Sucesso: resposta_fusion_330654516.txt
[2/1584] ✅ Sucesso: resposta_fusion_330657860.txt
[3/1584] ✅ Sucesso: resposta_fusion_330689295.txt
[4/1584] ✅ Sucesso: resposta_fusion_330696434.txt
[5/1584] ✅ Sucesso: resposta_fusion_330702487.txt
[6/1584] ✅ Sucesso: resposta_fusion_330706851.txt
[7/1584] ✅ Sucesso: resposta_fusion_330711777.txt
[8/1584] ✅ Sucesso: resposta_fusion_330711831.txt
[9/1584] ✅ Sucesso: resposta_fusion_330712784.txt
[10/1584] ✅ Sucesso: resposta_fusion_330715125.txt
[11/1584] ✅ Sucesso: resposta_fusion_330721592.txt
[12/1584] ✅ Sucesso: resposta_fusion_330728612.txt
[13/1584] ✅ Sucesso: resposta_fusion_330748919.txt
[14/1584] ✅ Sucesso: resposta_fusion_330753562.txt
[15/1584] ✅ Sucesso: resposta_fusion_330759425.txt
[16/1584] ✅ Sucesso: resposta_fusion_330765509.txt
[17/1584] ✅ Sucesso: resposta_fusion_330768762.txt
[18/158